## Experiment-05
### Amirreza Fosoul and Bithiah Yuan

In [15]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
import string
import csv
import re
import time
spark = SparkSession.builder.appName('ex5').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.functions import split, udf, desc, concat, col, lit
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType, DoubleType, StructType, StructField
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.window import Window
from pyspark.ml.linalg import SparseVector, VectorUDT, DenseVector
import scipy.sparse
from pyspark.ml.linalg import Vectors, _convert_to_vector, VectorUDT
import numpy as np
from pyspark.sql import SQLContext
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import math
import re
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel
from pyspark.mllib.feature import Word2Vec

sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [4]:
# Read user ratings into Dataframe
#user_df = spark.read.option("delimiter", ";").csv('./users_libraries.txt')
user_df = spark.read.option("delimiter", ";").csv('./example0.txt')
user_df = user_df.select(col("_c0").alias("userID"), col("_c1").alias("paperID"))

# df to be used in 4.4
sampled_users = user_df

user_df_pre = user_df
user_df_pre = user_df_pre.withColumn("paperID", split(col("paperID"), ",").cast(ArrayType(IntegerType())).alias("paperID"))
user_df = user_df.select("userID", f.split("paperID", ",").alias("papers"), f.explode(f.split("paperID", ",")).alias("paperID"))
user_df = user_df.drop("papers")

# Get a dataframe of the distinct papers
d_paper = user_df.select("paperID").distinct()

# Read in the stopwords as a list
with open('./stopwords_en.txt') as file:
    stopwordList = file.read().splitlines()

# Read in records of paper information
#w_df = spark.read.csv('./papers.csv')
w_df = spark.read.csv('./paper0.csv')
w_df = w_df.select("_c0", "_c13", "_c14")
w_df = w_df.select(col("_c0").alias("paperID"), col("_c13").alias("title"), col("_c14").alias("abstract"))
w_df = w_df.na.fill({'title': '', 'abstract': ''}) # to replace null values with empty string
# Get text from title and abstract
w_df = w_df.select(col("paperID"), concat(col("title"), lit(" "), col("abstract")).alias("words"))
#w_df.show()

# # Transform the distinct paperIDs dataframe to a list
# paper_list = list(d_paper.select('paperID').toPandas()['paperID'])
# # Map each distinct paper into int
# paper_list = list(map(int, paper_list))

In [20]:
################################### Conservative pre-processing ###################################################

# Extracting words from the papers and keeping "-" and "_"
tokenizer = RegexTokenizer(inputCol="words", outputCol="tokens", pattern="[a-zA-Z-_]+", gaps=False) 
# Built-in tokenizer
tokenized = tokenizer.transform(w_df)
tokenized = tokenized.select("paperID", "tokens")

# udf to remove "-" and "_" from the tokens
remove_hyphen_udf = udf(lambda x: [re.sub('[-|_]', '', word) for word in x], ArrayType(StringType()))
# Apply udf to the tokens
df = tokenized.withColumn('tokens', remove_hyphen_udf(col('tokens')))
# udf to remove words less than 3 letters
remove_short_words = udf(lambda x: [item for item in x if len(item) >= 3], ArrayType(StringType()))
# Apply udf to the tokens
df = df.withColumn('tokens', remove_short_words(col('tokens')))

conservative_df = df.rdd

conservative_df.collect()

# word2vec = Word2Vec()
# model = word2vec.fit(conservative_df)

# synonyms = model.findSynonyms('1', 5)

# for word, cosine_distance in synonyms:
#     print("{}: {}".format(word, cosine_distance))

[Row(paperID='12832332', tokens=['deepening', 'democracy', 'the', 'local', 'level', 'local', 'governance', 'bodies', 'lgbs', 'have', 'been', 'part', 'and', 'parcel', 'the', 'nepalese', 'people', 'for', 'more', 'than', 'five', 'decades', 'for', 'nearly', 'all', 'nepa', 'lese', 'people', 'the', 'most', 'familiar', 'public', 'organization', 'one', 'two', 'local', 'governance', 'bodies', 'village', 'development', 'committee', 'vdc', 'rural', 'areas', 'municipalities', 'urban', 'areas', 'and', 'the', 'district', 'development', 'committee', 'the', 'district', 'level', 'since', 'there', 'has', 'been', 'election', 'these', 'bodies', 'and', 'upon', 'completion', 'the', 'five', 'year', 'term', 'the', 'representatives', 'elected', 'july', 'are', 'being', 'run', 'and', 'managed', 'government', 'employees', 'the', 'government', 'funding', 'support', 'the', 'lgbs', 'has', 'however', 'been', 'increasing', 'trend', 'another', 'major', 'program', 'the', 'local', 'governance', 'and', 'community', 'devel

In [14]:
################################## Intensive pre-processing #######################################################

# Built-in function to remove stopwords from our custom list
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered" , stopWords=stopwordList)
df = remover.transform(df)
df = df.select("paperID", "filtered")

# Apply stemming with NLTK
# Built-in class from NLTK
ps = PorterStemmer()
# udf to apply stemming
stemming = udf(lambda x: [ps.stem(item) for item in x], ArrayType(StringType()))
# apply udf to tokens
df = df.withColumn('stemmed', stemming(col('filtered')))
df = df.select("paperID", "stemmed")

# Create a new df to store the paperID and stemmed tokens
paper_terms = df

# Explode/Split the tokens in the list for each paperID and get the distinct tokens
df = df.select("paperID", f.explode("stemmed").alias("tokens")).distinct().orderBy("paperID")

intensive_df = df

intensive_df.show()

+-------+----------+
|paperID|    tokens|
+-------+----------+
| 100088|   databas|
| 100088| algorithm|
| 100088|       way|
| 100088|  approxim|
| 100088|  stochast|
| 100088|    method|
| 100088|   analysi|
| 100088|   flexibl|
| 100088|     basic|
| 100088| implement|
| 100088|     gener|
| 100088|  properti|
| 100088|comparison|
| 100088|    search|
| 100088|      tool|
| 100088|     exist|
| 100088|   statist|
| 100088|     appli|
| 100088|    includ|
| 100088|  approach|
+-------+----------+
only showing top 20 rows



In [ ]:
# Assign count of 1 to each token w.r.t. the paperID since the tokens are distinct
df = df.groupBy("paperID","tokens").count()

# Get the number of distinct papers
num_papers = w_df.select("paperID").distinct().count()

# Get the value of ten percent of the number of papers
ten_percent = math.ceil(num_papers*.1)

# Create a new df with the tokens and count (without paperID)
df2 = df.select("tokens", "count")
# Count the number of papers containing the tokens
df2 = df2.groupBy("tokens").agg(f.collect_list("tokens").alias("duplicated_values"), f.sum("count").alias("count"))
# Filter out tokens that appeared in more than 10 percent of the papers
df2 = df2.drop("duplicated_values").orderBy((col("count")).desc()).filter(col("count") < ten_percent)
# Filter out tokens that appeared in less than 20 papers
# Limit the df to 1000 tokens
df2 = df2.filter(col("count") >= 20).limit(1000)
# Create a new df with terms and count
important_words = df2.select(col("tokens").alias("terms"), col("count"))
# Output the set of important words, T
#important_words.show()

In [4]:
# Function to call in udf
def unrated(papers):
    # Transform the list of distinct papers and the list of rated papers of each user to a set
    # Substract the two sets to get the list of unrated papers for each user
    # Transform back to list
    unrated = list(set(paper_list) - set(papers))
    
    return unrated


# udf to get a list of unrated papers with the length of rated papers for each user
get_unrated = udf(lambda x: unrated(x), ArrayType(IntegerType()))

# Add a new column of unrated papers for each user
unrated_df = user_df_pre.withColumn("unrated", get_unrated(user_df_pre.paperID))
unrated_df = unrated_df.drop("paperID")
unrated_df = unrated_df.withColumn("paperID", explode(unrated_df.unrated))
unrated_df = unrated_df.drop("unrated")

unrated_df.show()

+--------------------+--------+
|              userID| paperID|
+--------------------+--------+
|589b870a611c25fa9...| 9485312|
|589b870a611c25fa9...|14120961|
|589b870a611c25fa9...| 9676804|
|589b870a611c25fa9...|  604166|
|589b870a611c25fa9...| 2526216|
|589b870a611c25fa9...| 6804489|
|589b870a611c25fa9...| 1610761|
|589b870a611c25fa9...|10049547|
|589b870a611c25fa9...|11733005|
|589b870a611c25fa9...| 4186128|
|589b870a611c25fa9...|  970776|
|589b870a611c25fa9...|  118812|
|589b870a611c25fa9...|12306465|
|589b870a611c25fa9...| 5317666|
|589b870a611c25fa9...| 7152677|
|589b870a611c25fa9...| 6392873|
|589b870a611c25fa9...|13223978|
|589b870a611c25fa9...| 6967343|
|589b870a611c25fa9...|  255030|
|589b870a611c25fa9...| 8020025|
+--------------------+--------+
only showing top 20 rows



In [7]:
# Create a new df where each term is replaced by a unique index that takes a value from the range between 0 and |T| − 1
df = important_words.withColumn("row_num", row_number().over(Window.orderBy("count"))-1)
# Create a df to store the indices and the corresponding terms
terms_index_hash = df.select(col("row_num").alias("index"), "terms")
#terms_index_hash.show()

num_terms = terms_index_hash.select("terms").distinct().count()
#print(num_terms)

In [8]:
p_terms = paper_terms.select("paperID", f.explode("stemmed").alias("terms"))

# Join p_terms with the terms_index_hash to replace the terms with the indices
joined_df = terms_index_hash.join(p_terms, ["terms"])
joined_df = joined_df.drop("index")

# Create a new df to compute the term frequency vectors
tf_df = joined_df
tf_df = tf_df.groupby("paperID").agg(f.concat_ws(", ", f.collect_list(tf_df.terms)).alias("terms"))
tf_df = tf_df.withColumn("terms_", split(col("terms"), ",\s*").cast(ArrayType(StringType())).alias("terms"))
tf_df = tf_df.drop("terms")
# tf_df is now a df with a column of paperID and a column of lists of the terms (unexploded)
#tf_df.show()

cv = CountVectorizer(inputCol="terms_", outputCol="vectors")
model = cv.fit(tf_df)
vector_df = model.transform(tf_df)
vector_df = vector_df.select("paperID", col("vectors").alias("term_frequency_sparse"))

# Term frequency vector df
#vector_df.show()

In [9]:
########################################## TF-IDF with built-in function ##########################################

from pyspark.ml.feature import HashingTF, IDF, Tokenizer

idf = IDF(inputCol="term_frequency_sparse", outputCol="features")
idfModel = idf.fit(vector_df)
rescaledData = idfModel.transform(vector_df)
tf_idf_built_in = rescaledData.select("paperID", "features")

#tf_idf_built_in.show()

# create the user profile using the tf_idf dataframe and the users' library dataframe
user_profile = tf_idf_built_in.join(user_df, ["paperID"]).orderBy("userID").select('userId', 'features')

# convert the dataframe to RDD to sum up the tf_idf vector of each user and then convert back to dataframe
user_profile = user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])

user_profile = user_profile.select("userId", col("features").alias("user_profile"))

user_profile.show()


+--------------------+--------------------+
|              userId|        user_profile|
+--------------------+--------------------+
|1eac022a97d683eac...|[12.3235197123524...|
|589b870a611c25fa9...|[0.0,0.0,0.0,0.0,...|
|90f1a3e6fcdbf9bc5...|[0.0,0.0,24.05779...|
|0e3bcf22520d804fe...|[0.0,0.0,0.0,0.0,...|
+--------------------+--------------------+



In [9]:
# Convert dense vectors to sparse
def to_sparse(x):        
    # store the indices of non-zero elements
    nonzero_indices = np.nonzero(x)[0].tolist()
    # store the value of non-zero elements
    nonzero_counts = [num for num in x if num]
    # combine them to make a sparse vector
    sparse = SparseVector(num_terms, nonzero_indices, nonzero_counts)
    return sparse

to_sparse_udf = udf(lambda x: to_sparse(x), VectorUDT())

# Conver the feature vectors to sparse vectors
user_profile = user_profile.withColumn("user_profile", to_sparse_udf(col("user_profile")))

#user_profile.show()

# Join with the user profile with their unrated papers from unrated_df
df = unrated_df.join(tf_idf_built_in, ["paperID"]).join(user_profile, ["userID"])
df = df.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))
# df is now a dataframe with userID, paperID, user_profile, and paper_profile

#df.show()

# Select a user
df_selected = df.where(df.userID=="1eac022a97d683eace8815545ce3153f")

#df_selected.show()

## Exercise 4. 2 (Content-based recommendations: similarity metric)

In [ ]:
import numpy as np 

def cos_sim(u, p):
    result = (np.dot(u, p))/(np.linalg.norm(u) * np.linalg.norm(p))
    result = result.item()
    return result

compute_sim = udf(cos_sim, FloatType())

## Exercise 4. 3 (Content-based recommendations)

In [11]:
def cbrs(u, k):
    # Apply similarity metric to the user_profile and paper_profile
    sim_df = u.withColumn('Similarity', compute_sim(u.user_profile, u.paper_profile))
    # Partition by userID and order by the similarity in descending order
    window = Window.partitionBy(col("userID")).orderBy((col("Similarity")).desc())
    # Add row numbers to the rows and get the top-k rows
    sim_df = sim_df.select(col('*'), row_number().over(window).alias('row_number')).where(col('row_number') <= k)
    # Renaming
    get_r = sim_df.select("userID", "paperID", col("row_number").alias("rank"))
    cbrs_df = get_r.select("userID", "paperID")
    # un-explode, concatenate the recommended papers for each user
    cbrs_df = cbrs_df.groupby("userID").agg(f.concat_ws(", ", f.collect_list(cbrs_df.paperID)).alias("top_papers"))
    
    return cbrs_df

k = 5

user_rec = cbrs(df_selected, k).show(truncate=False)

#print("The top " + str(k) + " papers for " + str(user_rec.head()[0]) + " are " + str(user_rec.head()[1]))

+--------------------------------+-------------------------------------------+
|userID                          |top_papers                                 |
+--------------------------------+-------------------------------------------+
|1eac022a97d683eace8815545ce3153f|11733005, 8336239, 7010764, 2887105, 115945|
+--------------------------------+-------------------------------------------+



In [12]:
### LDA ###
from pyspark.ml.clustering import LDA

# just rename the column to use it with the built-in methods
termFrequencyVector = vector_df.select('paperId', col('term_frequency_sparse').alias('features'))
# Trains a LDA model
# set k=40 to have 40 different topics
lda = LDA(k=40)
model = lda.fit(termFrequencyVector)

# each topic is described by 5 terms
topics = model.describeTopics(5)

# print("The topics described by their top-weighted terms:")
#topics.show(truncate=False)

# Shows the result
# it shows the probabilty of each topic for each paper
transformed = model.transform(termFrequencyVector)
#transformed.show()

In [13]:
# creating a user profile based on the LDA results

lda_user_profile = transformed.join(user_df, ["paperID"]).orderBy("userID").select('userId', col('topicDistribution').alias('features'))

lda_user_profile = lda_user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])
lda_user_profile = lda_user_profile.select("userId", col("features").alias("lda_user_profile"))

lda_user_profile = lda_user_profile.withColumn("user_profile", to_sparse_udf(col("lda_user_profile")))

#lda_user_profile.show()

df_lda = unrated_df.join(transformed, ["paperID"]).join(lda_user_profile, ["userID"])

df_lda = df_lda.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))

#df_lda.show()

df_selected_lda = df_lda.where(df_lda.userID=="1eac022a97d683eace8815545ce3153f")

#df_selected_lda.show()

In [14]:
cbrs(df_selected_lda, 5).show(truncate=False)

+--------------------------------+--------------------------------------------+
|userID                          |top_papers                                  |
+--------------------------------+--------------------------------------------+
|1eac022a97d683eace8815545ce3153f|7164691, 9563857, 1363828, 11733005, 8336239|
+--------------------------------+--------------------------------------------+



## Exercise 4. 4 Sampling and data preparation

In [15]:
from pyspark.sql.functions import rand 

num_user = 2

# Order the users randomly and get n sampled users
sampled_users = user_df_pre.orderBy(rand()).limit(num_user)

# Get length of each list in a column
get_len_udf = udf(lambda x: len(x), IntegerType())

# Get library size for each user
sampled_users = sampled_users.withColumn("libSize", get_len_udf("paperID"))

# Get the size of the training set
get_train = udf(lambda x: int(x*0.8), IntegerType())

# Get size of the training set for each user
sampled_users = sampled_users.withColumn("trainSize", get_train("libSize"))

sampled_users.show()

+--------------------+--------------------+-------+---------+
|              userID|             paperID|libSize|trainSize|
+--------------------+--------------------+-------+---------+
|1eac022a97d683eac...|[3973229, 322433,...|    321|      256|
|589b870a611c25fa9...|[1283233, 1305474...|      8|        6|
+--------------------+--------------------+-------+---------+



In [16]:
# explode the paperIDs for each user
sampled_exploded = sampled_users.withColumn("paperID", explode(col("paperID")))

# Partion by userID and order them randomly
window = Window.partitionBy(col("userID")).orderBy(rand())

# Get row numbers
sampled_exploded = sampled_exploded.select(col('*'), row_number().over(window).alias('row_number'))

# Get the rows less than or equal to the training set size
# The rows will be different each time because of .orderBy(rand()) in the window function
training_df = sampled_exploded.where(col('row_number') <= col("trainSize"))
training_df = training_df.select("userID", "paperID").orderBy("userID")
#training_df.show()

# Get the test set by selecting the rows greater than the training size
test_df = sampled_exploded.where(col('row_number') > col("trainSize"))
test_df = test_df.select("userID", "paperID").orderBy("userID")
#test_df.show()

In [17]:
# create the user profile using the tf_idf dataframe and the users' library dataframe
training_user_profile = tf_idf_built_in.join(training_df, ["paperID"]).orderBy("userID").select('userId', 'features')

# convert the dataframe to RDD to sum up the tf_idf vector of each user and then convert back to dataframe
training_user_profile = training_user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])

# Same steps as above
training_user_profile = training_user_profile.select("userId", col("features").alias("user_profile"))

training_user_profile = training_user_profile.withColumn("user_profile", to_sparse_udf(col("user_profile")))

train_df = unrated_df.join(tf_idf_built_in, ["paperID"]).join(training_user_profile, ["userID"])

train_df = train_df.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))

train_df = unrated_df.join(tf_idf_built_in, ["paperID"]).join(training_user_profile, ["userID"])

train_df = train_df.select("userID", "paperID", "user_profile", col("features").alias("paper_profile")).orderBy("userID")

#train_df.show()

In [18]:
# creating a user profile based on the LDA results
training_lda_user_profile = transformed.join(training_df, ["paperID"]).orderBy("userID").select('userId', col('topicDistribution').alias('features'))

training_lda_user_profile = training_lda_user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])
training_lda_user_profile = training_lda_user_profile.select("userId", col("features").alias("lda_user_profile"))

training_lda_user_profile = training_lda_user_profile.withColumn("user_profile", to_sparse_udf(col("lda_user_profile")))

train_df_lda = unrated_df.join(transformed, ["paperID"]).join(training_lda_user_profile, ["userID"])

train_df_lda = train_df_lda.select("userID", "paperID", "user_profile", col("features").alias("paper_profile")).orderBy("userID")

#train_df_lda.show()

## Exercise 4. 5 (Off-line evaluation)

In [19]:
# a) Generate 10 recommendations

def castToArray(df, colName):
    dff = df.withColumn(colName, split(col(colName), ", ").cast(ArrayType(IntegerType())))
    return dff

k = 10

# TF-IDF
print("TF-IDF recommender")
tf_rec = cbrs(train_df, k).orderBy("userID")
# Cast the recommendations to a list of integers
tf_rec  = castToArray(tf_rec, "top_papers")
tf_rec.show(truncate=False)

print("LDA recommender")
# LDA
lda_rec  = cbrs(train_df_lda, k).orderBy("userID")
# Cast the recommendations to a list of integers
lda_rec  = castToArray(lda_rec, "top_papers")
lda_rec.show(truncate=False)

TF-IDF recommender
+--------------------------------+-----------------------------------------------------------------------------------------+
|userID                          |top_papers                                                                               |
+--------------------------------+-----------------------------------------------------------------------------------------+
|1eac022a97d683eace8815545ce3153f|[11733005, 7010764, 8336239, 115945, 7496675, 2887105, 8310458, 945604, 1305474, 9045137]|
|589b870a611c25fa99bd3d7295ac0622|[2887105, 7496675, 8336239, 9563857, 7010764, 1042553, 7164691, 9045137, 255030, 3010240]|
+--------------------------------+-----------------------------------------------------------------------------------------+

LDA recommender
+--------------------------------+------------------------------------------------------------------------------------------+
|userID                          |top_papers                                            

In [20]:
# Concatenate the test set into a list of integers
test_df_collected = test_df.groupby("userID").agg(f.concat_ws(", ", f.collect_list(test_df.paperID)).alias("paperID"))
test_df_collected = test_df_collected.withColumn("paperID", split(col("paperID"), ",\s*").cast(ArrayType(IntegerType())).alias("paperID")).orderBy("userID")

test_df_collected.show()

+--------------------+--------------------+
|              userID|             paperID|
+--------------------+--------------------+
|1eac022a97d683eac...|[3469193, 600359,...|
|589b870a611c25fa9...|   [1283233, 956315]|
+--------------------+--------------------+



In [21]:
joined_tf_test = test_df_collected.join(tf_rec, "userID")
joined_tf_test = joined_tf_test.select("userID", col("paperID").alias("test_set"), col("top_papers").alias("train_set"))

joined_lda_test = test_df_collected.join(lda_rec, "userID")
joined_lda_test = joined_lda_test.select("userID", col("paperID").alias("test_set"), col("top_papers").alias("train_set"))

In [22]:
def getHits(train, test):
    return list(set(train).intersection(test))

getHits_udf = udf(getHits, ArrayType(IntegerType()))
# TF-IDF
print("TF-IDF recommender")
tf_hits = joined_tf_test.withColumn('Hits', getHits_udf(joined_tf_test.train_set, joined_tf_test.test_set))
tf_hits = tf_hits.select("userID", "Hits")
tf_hits.show(truncate=False)

print("LDA recommender")
# LDA
lda_hits = joined_lda_test.withColumn('Hits', getHits_udf(joined_lda_test.train_set, joined_lda_test.test_set))
lda_hits = lda_hits.select("userID", "Hits")
lda_hits.show(truncate=False)

TF-IDF recommender
+--------------------------------+----+
|userID                          |Hits|
+--------------------------------+----+
|1eac022a97d683eace8815545ce3153f|[]  |
|589b870a611c25fa99bd3d7295ac0622|[]  |
+--------------------------------+----+

LDA recommender
+--------------------------------+----+
|userID                          |Hits|
+--------------------------------+----+
|1eac022a97d683eace8815545ce3153f|[]  |
|589b870a611c25fa99bd3d7295ac0622|[]  |
+--------------------------------+----+



### Since we could not get results for the hits, we modified the results to show our computations for the evaluation

In [23]:
########## example #####################

columns = ['userID', 'Hits']
vals = [("user0", "1, 2, 3"), ("user1", "4, 5, 6")]

ex = sqlContext.createDataFrame(vals, columns)
ex = castToArray(ex, "Hits")

ex_hits = ex.union(tf_hits)
ex_hits.show()

columns2 = ['userID', 'paperID']
vals2 = [("user0", "2, 3, 4, 1, 5"), ("user1", "5, 4, 6, 7, 8, 9, 10")]
ex2 = sqlContext.createDataFrame(vals2, columns2)
ex2 = castToArray(ex2, "paperID")

ex_test = ex2.union(test_df_collected)
ex_test.show()

+--------------------+---------+
|              userID|     Hits|
+--------------------+---------+
|               user0|[1, 2, 3]|
|               user1|[4, 5, 6]|
|1eac022a97d683eac...|       []|
|589b870a611c25fa9...|       []|
+--------------------+---------+

+--------------------+--------------------+
|              userID|             paperID|
+--------------------+--------------------+
|               user0|     [2, 3, 4, 1, 5]|
|               user1|[5, 4, 6, 7, 8, 9...|
|1eac022a97d683eac...|[3469193, 600359,...|
|589b870a611c25fa9...|   [1283233, 956315]|
+--------------------+--------------------+



In [24]:
joined_test_hits = ex_test.join(ex_hits, "userID")
joined_test_hits.show()

+--------------------+--------------------+---------+
|              userID|             paperID|     Hits|
+--------------------+--------------------+---------+
|               user1|[5, 4, 6, 7, 8, 9...|[4, 5, 6]|
|1eac022a97d683eac...|[3469193, 600359,...|       []|
|589b870a611c25fa9...|   [1283233, 956315]|       []|
|               user0|     [2, 3, 4, 1, 5]|[1, 2, 3]|
+--------------------+--------------------+---------+



In [29]:
num_user = 4

def hitSize_k(hits):
    return len(hits)/k

hitSize_k_udf = udf(lambda x: hitSize_k(x), FloatType())

def precisionK(df):
    df = df.withColumn("hitSize_k", hitSize_k_udf("Hits"))
    sumHits_k = df.select(f.sum("hitSize_k")).collect()[0][0]
    precision = (1/num_user)*sumHits_k
    return precision

print("The precision@" + str(k) + " for TF-IDF is: " + ("%.2f" % precisionK(ex_hits)))

The precision@10 for TF-IDF is: 0.15


In [30]:
def hitSize_testSize(hits, testSize):
    return len(hits)/len(testSize)

hitSize_testSize_udf = udf(hitSize_testSize, FloatType())

def recallK(df):
    df = df.withColumn("hitSize_testSize", hitSize_testSize_udf(df.Hits, df.paperID))
    sumHits_test = df.select(f.sum("hitSize_testSize")).collect()[0][0]
    recall = (1/num_user)*sumHits_test
    return recall

print("The Recall@" + str(k) + " for TF-IDF is: " + ("%.2f" % recallK(joined_test_hits)))

The Recall@10 for TF-IDF is: 0.26


In [54]:
def getPositionU(hits, test):
    if not hits:
        return 0.0
    else:
        return 1/test.index(hits[0])

getPositionU_udf = udf(getPositionU, FloatType())

def mrrK(df):
    df = df.withColumn("P_u", getPositionU_udf(df.Hits, df.paperID))
    sumP_u = df.select(f.sum("P_u")).collect()[0][0]
    mrr = (1/num_user)*sumP_u
    return mrr

print("The MRR@" + str(k) + " for TF-IDF is: " + ("%.2f" % mrrK(joined_test_hits)))

The MRR@10 for TF-IDF is: 0.33


### Since we could not get good results for the hits (empty), we computed the precision for k in {10, 20}
#### We could change the 21 in the range to 101 to compute the precision up till k = 100

In [61]:
num_user = 2

for k in range(10, 21):
    # TF-IDF
    tf_rec = cbrs(train_df, k).orderBy("userID")
    tf_rec  = castToArray(tf_rec, "top_papers")
    joined_tf_test = test_df_collected.join(tf_rec, "userID")
    joined_tf_test = joined_tf_test.select("userID", col("paperID").alias("test_set"), col("top_papers").alias("train_set"))
    tf_hits = joined_tf_test.withColumn('Hits', getHits_udf(joined_tf_test.train_set, joined_tf_test.test_set))
    tf_hits = tf_hits.select("userID", "Hits")
    test_hits =  test_df_collected.join(tf_hits, "userID")
    
    print("The precision@" + str(k) + " for TF-IDF is: " + ("%.2f" % precisionK(tf_hits)))
    print("The Recall@" + str(k) + " for TF-IDF is: " + ("%.2f" % recallK(test_hits)) + "\n")
   

The precision@10 for TF-IDF is: 0.00
The Recall@10 for TF-IDF is: 0.00

The precision@11 for TF-IDF is: 0.00
The Recall@11 for TF-IDF is: 0.00

The precision@12 for TF-IDF is: 0.00
The Recall@12 for TF-IDF is: 0.00

The precision@13 for TF-IDF is: 0.00
The Recall@13 for TF-IDF is: 0.00

The precision@14 for TF-IDF is: 0.00
The Recall@14 for TF-IDF is: 0.00

The precision@15 for TF-IDF is: 0.00
The Recall@15 for TF-IDF is: 0.00

The precision@16 for TF-IDF is: 0.00


KeyboardInterrupt: 

In [ ]:
for k in range(10, 21):
    # LDA
    lda_rec  = cbrs(train_df_lda, k).orderBy("userID")
    lda_rec  = castToArray(lda_rec, "top_papers")
    joined_lda_test = test_df_collected.join(lda_rec, "userID")
    joined_lda_test = joined_lda_test.select("userID", col("paperID").alias("test_set"), col("top_papers").alias("train_set"))
    lda_hits = joined_lda_test.withColumn('Hits', getHits_udf(joined_lda_test.train_set, joined_lda_test.test_set))
    lda_hits = lda_hits.select("userID", "Hits")
    test_hits_lda =  test_df_collected.join(lda_hits, "userID")
    
    print("The precision@" + str(k) + " for LDA is: " + ("%.2f" % precisionK(tf_hits)))
    print("The Recall@" + str(k) + " for LDA is: " + ("%.2f" % recallK(test_hits)) + "\n")
